<a href="https://colab.research.google.com/github/terahidro2003/ID2223_finetuning/blob/main/ID2223_fine_tuning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
## Install Unsloth

In [12]:
%%capture

!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

## Constants

In [13]:
import os 

if "google.colab" in str(get_ipython()):
  from google.colab import userdata
  TOKEN = userdata.get('HF_TOKEN')
else:
  !pip install python-dotenv
  from dotenv import load_dotenv
  load_dotenv()
  TOKEN = os.environ["HF_TOKEN"]

MODEL_NAME = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
hub_repo = "hellstone1918/test-model"

Defaulting to user installation because normal site-packages is not writeable


## Specify Quantizied Model 

In [14]:
!pip install unsloth
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Defaulting to user installation because normal site-packages is not writeable
  Using cached unsloth-2025.11.4-py3-none-any.whl (358 kB)
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA GeForce RTX 4070. Num GPUs = 1. Max memory: 11.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Adding LoRA Adapter

In [15]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# Data Preparation

## Download Dataset

In [16]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

## Convert Dataset Format to HuggingFace's Generic Format

In [17]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True, )

## Comparison of Formats

In [18]:
dataset[5]["conversations"]

[{'content': 'How do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?',
  'role': 'user'},
 {'content': 'Astronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.',
  'role': 'assistant'}]

In [19]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAstronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.<|

# Training

In [20]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
import multiprocessing

sft_config = SFTConfig(
    output_dir       = "outputs",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps     = 5,
    max_steps        = 60,
    learning_rate    = 2e-4,
    fp16             = not is_bfloat16_supported(),
    bf16             = is_bfloat16_supported(),
    logging_steps    = 1,
    optim            = "adamw_8bit",
    weight_decay     = 0.01,
    lr_scheduler_type= "linear",
    seed             = 3407,

    # Checkpoint saving and pushing config
    push_to_hub = True,
    hub_model_id = hub_repo,
    hub_token=TOKEN,
    save_strategy="steps",
    save_steps       = 10,
    save_total_limit = 5,
    hub_strategy="all_checkpoints",

    report_to        = "none",

    dataset_num_proc = multiprocessing.cpu_count(),
    packing          = False,
)

trainer = SFTTrainer(
    model              = model,
    tokenizer          = tokenizer,
    train_dataset      = dataset,
    args               = sft_config,
    dataset_text_field = "text",
    max_seq_length     = max_seq_length,
    data_collator      = DataCollatorForSeq2Seq(tokenizer = tokenizer),
)

In [21]:
import os
import re
from huggingface_hub import HfApi, snapshot_download

def get_latest_hf_checkpoint(repo_id: str, cache_dir: str | None = None) -> str | None:
    """
    Returns local path to the latest checkpoint-* directory from a Hub repo,
    or None if the repo doesn't exist or has no checkpoints.
    """
    api = HfApi()
    try:
        api.repo_info(repo_id, repo_type="model")
    except:
        # Repo does not exist on the Hub -> first training ever
        return None

    # Repo exists – download files locally (or reuse cached snapshot)
    local_repo_path = snapshot_download(
        repo_id,
        repo_type="model",
        local_dir=cache_dir,                # e.g. "hf_repo_cache" or None
        local_dir_use_symlinks=False,
    )

    # Find checkpoint-* subdirectories
    ckpts: list[tuple[int, str]] = []
    for name in os.listdir(local_repo_path):
        full = os.path.join(local_repo_path, name)
        m = re.match(r"checkpoint-(\d+)", name)
        if m and os.path.isdir(full):
            step = int(m.group(1))
            ckpts.append((step, full))

    if not ckpts:
        # Repo exists but no checkpoints yet (maybe only final model)
        return None

    # Return path of checkpoint with highest step number
    _, latest_ckpt_path = max(ckpts, key=lambda t: t[0])
    return latest_ckpt_path


In [22]:
from unsloth.chat_templates import train_on_responses_only

#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)
latest_ckpt = get_latest_hf_checkpoint(hub_repo)
if latest_ckpt is None:
    print("No checkpoints on Hub – starting training from scratch.")
    trainer_stats = trainer.train()
else:
    print(f"Found checkpoint on Hub: {latest_ckpt}")
    trainer_stats = trainer.train(resume_from_checkpoint=latest_ckpt)

GPU = NVIDIA GeForce RTX 4070. Max memory = 11.994 GB.
5.32 GB of memory reserved.


The model is already on multiple devices. Skipping the move to device specified in `args`.


No checkpoints on Hub – starting training from scratch.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.779700
2,0.839400
3,1.081300
4,0.892900
5,0.768100
6,0.941800
7,0.622000
8,1.002300
9,0.860100
10,0.760600


## Show Stats

In [23]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

239.8396 seconds used for training.
4.0 minutes used for training.
Peak reserved memory = 6.867 GB.
Peak reserved memory for training = 1.547 GB.
Peak reserved memory % of max memory = 57.254 %.
Peak reserved memory for training % of max memory = 12.898 %.


In [24]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "How would you balance the chemical equation for the reaction between ammonia (NH3) and oxygen (O2), which produces nitrogen dioxide (NO2) and water (H2O)?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


The balanced chemical equation for the reaction between ammonia (NH3) and oxygen (O2) is:

2 NH3 + 3 O2 → 2 NO2 + H2O<|eot_id|>


# Save the Model

In [36]:
!uv venv .venv
!source .venv/bin/activate --clear
!uv pip install "unsloth" "gguf" "protobuf" "sentencepiece" "mistral_common"

model.push_to_hub_gguf(hub_repo, tokenizer, quantization_method = "q4_k_m")

Using CPython 3.11.14
Creating virtual environment at: .venv
? A virtual environment already exists at `.venv`. Do you want to replace it? [y/n] › yes

hint: Use the `--clear` flag or set `UV_VENV_CLEAR=1` to skip this prompt
Audited 5 packages in 1.73s
Unsloth: Converting model to GGUF format...
Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /home/hellstone/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:13<00:00, 36.62s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:29<00:00, 14.86s/it]


Unsloth: Merge process complete. Saved to `/tmp/unsloth_gguf_u46cmaol`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['Llama-3.2-3B-Instruct.BF16.gguf']
Unsloth: [2] Converting GGUF bf16 into q4_k_m. This might take 10 minutes...
Unsloth: Model files cleanup...
Unsloth: All GGUF conversions completed successfully!
Generated files: ['Llama-3.2-3B-Instruct.Q4_K_M.gguf']
Unsloth: example usage for text only LLMs: llama-cli --model Llama-3

Processing Files (0 / 1): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 2.02GB / 2.02GB, 32.0MB/s  
New Data Upload: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.01GB / 2.01GB, 32.0MB/s  


Uploading config.json...
Uploading Ollama Modelfile...
Unsloth: Successfully uploaded GGUF to https://huggingface.co/hellstone1918/test-model
Unsloth: Cleaning up temporary files...


'hellstone1918/test-model'